In [1]:
%load_ext autoreload
%autoreload 2

import os
import matplotlib.pyplot as plt

# Change working directory to project root
os.chdir('../../')

import torch
import numpy as np
from pathlib import Path
from tqdm import tqdm
import json
from hmr4d.utils.wis3d_utils import make_wis3d
from hmr4d.utils.wis3d_utils import add_joints22_motion_as_lines
from hmr4d.dataset.rich.rich_utils import get_w2az_sahmr, parse_seqname_info
from hmr4d.utils.geo_transform import apply_T_on_points

In [2]:
wis3d = make_wis3d(name='rich_joints3d')

In [3]:
# Load joints3d-motion
rich_joints3d = np.load("inputs/RICH/hmr4d_support/joints3d/joints3d_smpl_v2.npy", allow_pickle=True).item()
rich_joints3d = {k: torch.from_numpy(v) for k, v in rich_joints3d.items()}
# add_joints22_motion_as_lines(rich_joints3d[seq_names[20]], wis3d, "joints22_w")

# Setup Meta for getitem(i)
meta = []
length = 120
rich_fps = 30
for seq_name in rich_joints3d.keys():
    total_length = rich_joints3d[seq_name].shape[0]
    # Rules: skip first 1 seconds (30frames), sample interval is 0.5 second
    for start_frame in range(rich_fps, total_length - length, int(rich_fps*0.5)):
        end_frame = start_frame + length
        meta.append((seq_name, start_frame, end_frame))

# Transformation
w2az = get_w2az_sahmr()  # scan_name: tensor 4,4
seqname_info = parse_seqname_info()
seqname_to_scanname = {k: v[0] for k, v in seqname_info.items()}


In [4]:
i = 42
seq_name, start_frame, end_frame = meta[i]

# Get 120 frames of motion
motion = rich_joints3d[seq_name][start_frame:end_frame]  # (120, 22, 3)
# add_joints22_motion_as_lines(motion, wis3d, "sample_c1")

# Transform to az
scan_name = seqname_to_scanname[seq_name]

motion_az = apply_T_on_points(motion, w2az[scan_name][None])
add_joints22_motion_as_lines(motion_az, wis3d, "sample_az")

In [9]:
from hmr4d.utils.geo_transform import compute_T_ayf2az

T_az2ayf = compute_T_ayf2az(motion_az[None, 0], inverse=True)[0]  # (4, 4)
motion_ayf = apply_T_on_points(motion_az, T_az2ayf[None])  # (F, 22, 3)
add_joints22_motion_as_lines(motion_ayf, wis3d, "sample_ayf")

In [10]:
# import json
# from pathlib import Path

# # Decide scene_name
# LectureHall_chair_seqs = [  # others are LectureHall_yoga
#     "LectureHall_018_wipingtable1",
#     "LectureHall_020_wipingchairs1",
#     "LectureHall_003_wipingchairs1",
#     "LectureHall_018_wipingchairs1",
#     "LectureHall_018_wipingspray1",
#     "LectureHall_020_wipingtable1",
#     "LectureHall_019_wipingchairs1",
# ]

# scene_name = seq_name.split("_")[0]
# if scene_name == "LectureHall":
#     scene_name += "_chair" if seq_name in LectureHall_chair_seqs else "_yoga"

# # Get cam2world
# cam2world_dir = Path("hmr4d/dataset/rich/multicam2world")
# cam2scan = json.load((cam2world_dir / f"{scene_name}_multicam2world.json").open())

# rot_mat = torch.tensor(cam2scan['R'])
# translation = torch.tensor(cam2scan['t'])

# print(motion.shape)
# motion_align = motion @ rot_mat[None] + translation[None]